# Rsync files in GCS

Rsync files in GCS from bucket to bucket

Import modules, inlcuding 'sourcing' py file with functions

In [1]:
from google.cloud import storage
import subprocess
import pandas as pd

# import multiprocessing as mp
# from itertools import repeat

%run -m file_move

Set variable names, and generate list of files to rename

In [2]:
# storage_client = storage.Client(project = "ggn-nmfs-usamlr-dev-7b99")
bucket_source    = "amlr-imagery-proc-dev"
file_prefix    = "gliders/2022/amlr08-20220513/shadowgraph/images/Dir"
file_substr    = "/regions/"

bucket_destination = "amlr-gliders-imagery-proc-dev"

In [3]:
file_list_orig = list_blobs_with_prefix(
    bucket_source, file_prefix, file_substr=file_substr)    

In [4]:
print(f"there are {len(file_list_orig)} files with {file_substr} " +
      f"with the prefix {bucket_source}/{file_prefix}")
print("First 10 files:") 
for i in file_list_orig[0:9]:
    print(i)

there are 750264 files with /regions/ with the prefix amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir
First 10 files:
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out0.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out1.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out2.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out3.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out4.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out5.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510-213508-001.jpg-out6.jpg
gliders/2022/amlr08-20220513/shadowgraph/images/Dir0000/regions/SG01 20220510

In [ ]:
# lengths_list = (lambda x:[len(i) for i in x])(file_list_orig)
# pd.Series(lengths_list).value_counts()

Create new file list

In [5]:
# dir_list = [i[:-1] for i in file_list_orig if len(i) <= 70]
dir_list = [i for i in file_list_orig if len(i) <= 70]
# print(f"There were {len(file_list_orig) - len(file_list)} files removed, from original {len(file_list_orig)}. ")
# print(f"There are {len(file_list)} files remaining")
print(f"There are {len(dir_list)} directory paths")

There are 48 directory paths


In [8]:
rsync_list_source = [f"gs://{bucket_source}/{i[:-1]}" for i in dir_list]
rsync_list_source = rsync_list_source[32:]

for i in rsync_list_source:
    print(i)

gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0032/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0033/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0034/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0035/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0036/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0037/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0038/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0039/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0040/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0041/regions
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0042/regions

## Create rsync arguments and run commands

Run all in single for loop, because multithreading crashes

In [ ]:
# rsync_list_destination = []
# for i in dir_list:
#     i_orig = i
#     i = i.replace("gliders/2022", "FREEBYRD/2023")
#     i = i.replace("/shadowgraph/", "/")
#     i = i.replace("/regions/", "")
#     i = i.replace("/images/", "/regions/")
#     rsync_list_destination.append(build_dest(i, bucket_destination))

# for j in rsync_list_destination:
#     print(j)

In [9]:
def rsync_build_run(src, bucket_name_src, bucket_name_dest, print_only=False):
    # i_orig = i
    x = src
    x = x.replace(f"gs://{bucket_name_src}/gliders/2022", 
                  f"gs://{bucket_name_dest}/SANDIEGO/2022")
    x = x.replace("/shadowgraph/", "/")
    x = x.replace("/regions", "")
    x = x.replace("/images/", "/regions/")
    dest = x

    run_list = ["gcloud", "storage", "rsync", "-r", src, dest]
    
    if print_only:
        return(" ".join(run_list))
    else:
        # return("tmp")
        run_out = subprocess.run(run_list, capture_output=True)
        if run_out.returncode != 0:
            print(f'Error')
            print(f'ARGS:\n{run_out.args}')
            print(f'STDERR:\n{run_out.stderr}')
            return(f'ERROR: {run_out.args}')
        else:
            return(f'Success: {run_out.args}')

In [10]:
for src in rsync_list_source:
    print("-------------------------------------------------")
    print(src)
    rsync_out = rsync_build_run(src, bucket_source, bucket_destination)
    print(rsync_out)

-------------------------------------------------
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0032/regions
Success: ['gcloud', 'storage', 'rsync', '-r', 'gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0032/regions', 'gs://amlr-gliders-imagery-proc-dev/SANDIEGO/2022/amlr08-20220513/regions/Dir0032']
-------------------------------------------------
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0033/regions
Success: ['gcloud', 'storage', 'rsync', '-r', 'gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0033/regions', 'gs://amlr-gliders-imagery-proc-dev/SANDIEGO/2022/amlr08-20220513/regions/Dir0033']
-------------------------------------------------
gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0034/regions
Success: ['gcloud', 'storage', 'rsync', '-r', 'gs://amlr-imagery-proc-dev/gliders/2022/amlr08-20220513/shadowgraph/images/Dir0034

### Multithreading
Multithreading crashes. Lol. Probably because `gcloud storage` is already trying to do multithreading things?

To take advantage of multiple cores anmd make this functionality robust to files already being there, it made sense to generate 'gcloud storage rsync' commands, and run those via multiprocessing.

NOTE: the function is not totally generalize, i.e. replacement strings will need to be updated for different operations.

In [ ]:
# # Multithreading crashes. Lol
# numcores = mp.cpu_count()
# print(f"Running with {numcores} cores")

# with mp.Pool(numcores) as pool:
#     out_list = pool.starmap(
#         rsync_build_run, 
#         zip(rsync_list_source, repeat(bucket_source), 
#             repeat(bucket_destination))
#     )

# for i in out_list:
#     print(i)

### Sanity checks

See comparisons file for sanity checks

In [12]:
# prefix_dest = "SANDIEGO/2022/amlr08-20220513/regions/Dir"
# file_list_dest = list_blobs_with_prefix(bucket_destination, prefix_dest)    
# print(f"there are {len(file_list_dest)} files " +
#       f"with the prefix {bucket_destination}/{prefix_dest}")

# lengths_list = (lambda x:[len(i) for i in x])(file_list_dest)
# pd.Series(lengths_list).value_counts()